# MovieLens 100K recommendation
MovieLens 100K のデータセットに対し、 
recommendation のアルゴリズムの1つ、 SVD
を実装した。

## 1. SVDアルゴリズムの説明

### recommedation アルゴリズムの概要

- 背景: 最近は情報過多社会。ユーザーにとって最適な商品（アイテム）の情報を探すのは大変。
- 目的: そこで各ユーザーが好む最適な商品を、全ての商品から選出する必要がある。
- 入力: ユーザーの行動履歴・評価 | アイテムの特徴 etc
- 出力: ユーザーの評価が高い・選ぶであろうアイテム | 似たようなアイテム

### recommedation アルゴリズムの分類

- **CF Collaborative Filetering**
    - memoery-based | Nearest Neighbor
        - user-user | user-based
        - item-item | item-based
    - **model-based**
        - **MF Matrix Factorization**
            - SVD
            - SVD++
            - NMF
        - clustering | regression | 確率モデル | topic model | Bayesean Network model | restricted Boltzmann machine | time series model
- Content-based
- Hybrid = CF + Content(|Demogarphic)
    - FM
    - VBPR

今回は、 CF > model-based > MF > SVD を実装した。

### CF vs Content-based
- **CF**
    - 入力: あるユーザーによる 商品のrating, 商品の購買・閲覧履歴 (e.g. 購入した服の5段階☆評価、ある服のページの閲覧、購入入したかしないか）
    - 出力: ユーザーの評価が高い・買うであろうアイテム (e.g. )
- Content-based
    - 入力: アイテムの特徴 (e.g. 服の画像) 
    - 出力: 似たようなアイテム (e.g. 見た目が似たような服)

### memory-based vs model-based

CF では、下のFig.1のような　**(ユーザー数) x (アイテム数) の形の user-item matrix** をデータとして扱う。
また、この行列は **rating matrix** とも言う。

![](https://i.imgur.com/bmW79NS.png)

Fig.1: user-item matrix rating

行がuser, 列がitem　を表し、
行列の各要素が、ある user による、ある item の 評価値である。

行列の要素は **1~5段階評価** などがよく使われるが、
評価ではなく、そのitemページの閲覧や、そのitemの購入をしたかしないか
の **バイナリ（0/1）** でもよい。

### memory-based

memory-based は、各行|列を user|item を表すベクトルとしてとらえる。
そして、**ベクトル間の内積(cosine)**などで、 user|item 間の類似度を計算し、
特定の user に対し、オススメの item を recommend する。

**user-based** だと、user ベクトルの類似度を計算し、
推薦をしたい user A が、まだ購入していない item のうち、
A と似ている user B, C が高評価しているアイテムを推薦する。

**item-based** では、 item ベクトルの類似度を計算し、
A が高評価している item と似ている item を推薦する。

欠点:
- user-item matrix は sparse (0ばかり) であることが多く、良い精度で recommend できない。
- recommendation 時に毎回、類似度を計算しなくてはならないので、 recommedation に時間がかかる。

### model-based

user-item matrix をそのまま推薦に用いるのではなく、
一度、 **統計モデル** を学習させてから、推薦する手法。

memory-based の欠点に対応している。
- user による評価値の推定 → sparse 性への対応
- 学習に時間がかかるが、一度学習してしまえば、推薦時間は短い

### MF Matrix Factorization
user-item matrix における各 user|item ベクトルを
潜在的な特徴量での表現に変換し、欠損している評価値を推定する手法。

具体的には、 user-item matrix R の形が $m \times n$ だとし、
user u による item i の評価値を $r_{u,i}$ とすると、
user u 、 item i を表す特徴量ベクトルは、

$\boldsymbol{r_u} = (r_{u1}, r_{u2}, ..., r_{un})^T$

shape: (n, 1)

$\boldsymbol{r_i} = (r_{1i}, r_{2i}, ..., r_{mi})$

shape: (1, m)

で表せる。
これらのベクトルを、$f$ （任意）次元の特徴量で表現し、
それぞれを **user|item factor** とする。

$\boldsymbol{x_u} = (x_{u1}, x_{u2}, ..., x_{uf})$

$\boldsymbol{y_i} = (y_{i1}, y_{i2}, ..., y_{if})$

これらを user|item ごとに縦に並べた行列を $X, Y$ とすると、 shape は、

- X: (m, f)
- Y: (n, f)

となる。

ここで、真のR (欠損値がないもの) は、
X, Yの積で表されると仮定する。
すなわち、

$R \approx XY^T$

となる。

この仮定を置く理由は、以下の直観的な背景からである。

user および item は、 f 個の潜在的な特徴量を持っていると考える。
たとえば、

- user ベクトルの視点: この user が SF をどれくらい好きかを表す数値
- item ベクトルの視点: この movie がどれくらい、 SF に当てはまるかの数値

などが潜在的な特徴量として考えられる。

userの好みのジャンルとitemのジャンルが近ければ、
そのuserによる、そのitemの評価は高くなることが直観的に考えられる。

また、userの潜在特徴ベクトルと、itemの潜在特徴ベクトルの内積は
両ベクトルが似ているほど高い値を示す。
つまり、先の、 ジャンルの好み と 評価値 の関係は、
この 潜在特徴ベクトル と その内積の値 の関係ににている。
なので、評価値は、各user、itemの潜在特徴ベクトルの内積で表せると仮定する。

$r_{u,j} \approx \boldsymbol{x_u}\boldsymbol{y_i}^T$

これを行列でまとめて表現すると、先ほどの仮定に一致する。

$R \approx XY^T$

また、この仮定は $R$ を
2つの行列 $X, Y$ に分解しているので、
行列分解 Matrix Factorizationとよぶ。

k は通常、 user 数 m、item 数 n よりも少なくする。
そうすることで、 sparse だった R を、
より dense な X, Y で表現できる。

### SVD

MF のアルゴリズムの代表の1つとして、
**SVD Singular Value Decomposition** があり、
今回はこれを実装した。
通常の数学の SVD では、 3つの行列に分解するが、
MF 用に 2つの行列に分解する点で異なる。

R の近似 $XY^T$ が、
真の $R$ に近づけばいいので、
これらの誤差を目的関数とする最小化問題を解けば良い。

### 最適化手法
上述の最適化問題を解く解法は以下の 2種類が代表的である。

- **SGD Stochastic Gradient Descent** (MF専用で少し特殊)
    - 数値的解法
    - 利点: 評価済のみを計算、 sparse な行列に強い
    - 欠点： 並列計算ができない
- ALS Alternative Least Square
    - 解析的解法
    - 利点: 並列計算が得意、データ数が多い|dense な行列に強い
    - 欠点: sparse な行列だと精度が良くない

後述するが、今回のような rating data は explicit data といい、 
行動履歴などのデータ (implicit) と違い、
自動でたまらず、**行列が sparse** であることが多い。

なので、今回、両方を実装したが、 SGD のほうが精度が良かった。
説明においては、 **SGD** に焦点を当てる。

### SGD (MF専用)
ALSに対し、SGDでは与えられた (欠損していない) 
評価値のみの誤差を使って最適化を行う。
式で表すと、

$
min. \sum_{u,i \in \{(u,i)|r_{u,i} \neq 0\}}(r_{u,i}-x_u y_i^T)^2 
+ \lambda(\|x_u\|^2 + \|y_i\|^2)
$

なお、学習パラメータは $X,Y$ であり、 
$\lambda$ は過学習を防ぐ正則化項の係数である。

上記の目的関数の勾配を用いて、パラメータ $X, Y$ を update していく。
更新式は以下。

$e_{u,i} = (r_{ui} - x_u y_i^T)$

$x_u \leftarrow x_u - \eta (- e_{u,i}y_i + \lambda x_u)$

$y_i \leftarrow y_i - \eta (- e_{u,i}x_u + \lambda y_i)$

- $e$ : 予測誤差
- $\eta$ : learning rateであり、後の括弧の中が目的関数の勾配である。

各、評価済みの user, item の組 $(u,i)$ について
この update を1回ずつ行っていく。

また、すべての組についての更新が終わったら、これを1epochとし、
学習時の引数として指定したepoch回数分、これを繰り返す。

また、今回は、学習パラメタに bias を追加した。

$R \approx XY^T + b^u + b^i + b$

- $b^u = (b^u_1, b^u_2, ..., b^u_m)$
    - shape: (m,1)
- $b^i = (b^i_1, b^i_2, ..., b^i_n)^T$
    - shape: (1,n)
- $b = mean(R_{\neq 0})$ (固定)

で近似する。
ただし、shapeにおいて長さが1となっている次元は、
最大数のものに合わせるようにconcatenateする
(e.g. (m,1)→(m,n)、axis=1の方向に同じベクトルを並べる。)

bias を導入することで、
辛口|甘口 user や、
大人気|不人気 item による評価の偏りを考慮できる。

たとえば、辛口でれば、user-bias のその user の要素は負の値をとり、
その user の評価が全 item において低くなるように調整できる。

上述の説明では、biasを加えると、SVDの本質がわかりにくくなるため、あえて省いた。
目的関数も、勾配の式も、導出の方法は上述同様。

#### 評価
MovieLensの trainting data u1.base、test data u1.test において、
u1.baseで学習させ、u1.testで評価した。

評価指標は直観的にわかりやすい RMSE Root Mean Squared Error を用いた。
評価値が1であれば、各予測評価値が真の観測評価値よりも平均して1ほどずれている
と考えればよい。


## 3. SVDの改善点

### explicit vs implicit data

- **explicit data**: user に恣意的に行ってもらった rating
    - e.g. 今回の movielens のような5段階で item を rating してもらったデータ
    - 短所: user に rating してもらうので、 user に負担がかかり、データを集めにくい。 data を集められない → recommend できない → user が集まらない → data が集まらない → ... という悪循環に陥る可能性がある。 （cold-start problem)
- **implicit data**
    - e.g. 購入回数、動画の再生時間、ページ閲覧回数などの行動履歴
    - 長所: 簡単に集められる。
    - 短所: rating が 正確でない場合がある。
        - rating が 0 だと、未評価なのか不支持なのかが曖昧。
        - 自分はその商品が嫌いだが、他人のために買って上げた場合
        - TV show の　recommend において、TV をつけっぱなしで寝てしまって、見ていないのに、再生時間が長い場合 etc
    
上述の SVD では explicit のみにしか対応していないので、 cold-start 問題に直面する可能性がある。

なので、改善策として、以下の2つのアプローチが考えられる。
- **implicit data** を用いる。
- **hybrid** (CF + content-based)

### SVD++

- SVD の **implicit data に対応** した version。
    - explicit より多くのデータを扱えるので、一般的に SVD より **精度が高く** なる。([Hu et al. 2008](http://yifanhu.net/PUB/cf.pdf))
- confidencの導入
    - implicit data では **rating が正確でない** ことがあるが、rating の値が高ければ高いほど、その rating は信用できると仮定をおいた。
    - そして、その信用度=confidence で **各 rating の重み付け** をし、より confidence の高い rating の誤差が、より小さくなるように学習する。
- ALS を用いた計算量の削減
    - explicit data に対し、 data 数が多くなる。
    - SVD では通常のSGDではなく並列計算ができないので、遅い。
    - そこで、SGDような数値的解法ではなく、解析的に局所最適化を繰り返すALS(Alternative Least Square)を用いることで、計算量を減らした。

### VBPR: Visual Bayesian Presonalized Ranking
- MF に CNN による画像特徴量を加えて、 rating matrix を予測するモデル([He et al. 2015](https://arxiv.org/pdf/1510.01784.pdf))
- さらに cold-start に強い。